In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [34]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [35]:
%load_ext rpy2.ipython

In [36]:
%R library(lme4)

'lmerTest','lme4','Matrix',...,'datasets','methods','base'


In [40]:
# Load the data from ../analysis/ENZYMES/graphs_{i}.npy
# where i is the index of the graph
def load_data(i):
    return np.load('../analysis/emp-hgpsl/ENZYMES/graphs_{}.npy'.format(i), allow_pickle=True)

In [41]:
def get_cna_metrics():

    # list of graph metrics for each dataset
    all_graph_metrics = []
    datasets = [load_data(i) for i in range(5)]
    
    for dataset in datasets:
        graph_metrics = []
        for graph in dataset:

            graph_metrics.append([
                                  graph.number_of_nodes(),
                                  nx.graph_number_of_cliques(graph),
                                  max(dict(graph.degree).values()),
                                  min(dict(graph.degree).values()),
                                  np.mean(list(dict(graph.degree).values())),
                                  #nx.degree_assortativity_coefficient(graph),
                                  nx.density(graph),
                                  np.mean(list(dict(graph.degree).values())),
                                  nx.average_clustering(graph),
                                  graph.graph['graph_id'],
                                  graph.graph['prediction'],
                                  graph.graph['label'],
                                  graph.graph['confidence'],
                                  graph.graph['correct'],
                                  graph.graph['loss'],
                                  graph.graph['replication']
                                  ])
        all_graph_metrics.append(graph_metrics)
    
    return all_graph_metrics

In [42]:
def rename_columns(df):
    # maps the index of graph_metrics to the metric name (needed to label automatically in visualization)
    index_to_metric = {0: 'Number_of_Vertices',
                       1: 'Number_of_Cliques',
                       2: 'Maximum_Degree',
                       3: 'Minimum_Degree',
                       4: 'Average_Degree',
                       5: 'Density',
                       6: 'Average_Neighbor_Degree',
                       7: 'Average_Clustering_Coefficient',
                       8: 'Graph_ID',
                       9: 'Prediction',
                       10: 'Label',
                       11: 'Confidence',
                       12: 'Correct',
                       13: 'Loss',
                       14: 'Replication',
                       }

    df = df.rename(columns=index_to_metric)
    return df

In [43]:
# build a dataframe for each dataset
def build_dataframe(all_graph_metrics):
    df_list = []
    for graph_metrics in all_graph_metrics:
        df = pd.DataFrame(graph_metrics)
        df = rename_columns(df)
        df_list.append(df)
    return df_list

In [44]:
df_list = build_dataframe(get_cna_metrics())

In [45]:
architecture = "emp-hgpsl"

In [46]:
# concatenate the dataframes
df = pd.concat(df_list, ignore_index=True)

# save the dataframe
# df.to_csv(f'../analysis/{architecture}/ENZYMES/df.csv', index=False)

In [47]:
# load the dataframe
# df = pd.read_csv(f'../analysis/{architecture}/ENZYMES/df.csv')

In [48]:
df['system'] = architecture

In [49]:
df.head().T

,0,1,2,3,4
Number_of_Vertices,10,50,34,33,52
Number_of_Cliques,5,74,39,37,68
Maximum_Degree,5,5,6,6,6
Minimum_Degree,2,2,2,3,2
Average_Degree,4,3.56,3.70588,4.60606,3.76923
Density,0.444444,0.0726531,0.112299,0.143939,0.0739065
Average_Neighbor_Degree,4,3.56,3.70588,4.60606,3.76923
Average_Clustering_Coefficient,0.74,0.156667,0.282353,0.475758,0.204487
Graph_ID,0,1,2,3,4
Prediction,1,0,2,5,5


In [57]:
md = smf.mixedlm("Loss ~ Replication", df, groups=df["Graph_ID"], re_formula="~Replication")
free = sm.regression.mixed_linear_model.MixedLMParams.from_components(
    np.ones(5), np.eye(5)
)
mdf = md.fit(free=free, method=["lbfgs"])
print(mdf.summary())

/home/bjoern/miniconda3/envs/pymer4/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                      Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          Loss     
No. Observations:          450              Method:                      REML     
No. Groups:                90               Scale:                       0.2124   
Min. group size:           5                Log-Likelihood:              -493.9282
Max. group size:           5                Converged:                   Yes      
Mean group size:           5.0                                                    
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                1.764    0.085 20.766 0.000  1.597  1.930
Replication[T.2]                        -0.015    0.072 -0.214 0.831 -0.156  0.125
Replication[T.3]           

In [55]:
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Mixed Linear Model Regression Results
==================================================================================
Model:                     MixedLM          Dependent Variable:          Loss     
No. Observations:          450              Method:                      REML     
No. Groups:                90               Scale:                       0.1490   
Min. group size:           5                Log-Likelihood:              -454.5579
Max. group size:           5                Converged:                   No       
Mean group size:           5.0                                                    
----------------------------------------------------------------------------------
                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------
Intercept                                1.764    0.075 23.533 0.000  1.617  1.911
Replication[T.2]                        -0.015    0.068 -0.226 0.822 -0.148  0.118
Replication[T.3]                         0.011    0.065  0.170 0.865 -0.117  0.139
Replication[T.4]                         0.165    0.141  1.172 0.241 -0.111  0.440
Replication[T.5]                         0.033    0.076  0.435 0.664 -0.116  0.182
Group Var                                0.357                                    
Group x Replication[T.2] Cov             0.052    0.113                           
Replication[T.2] Var                     0.116                                    
Group x Replication[T.3] Cov            -0.005    0.106                           
Replication[T.2] x Replication[T.3] Cov  0.027    0.068                           
Replication[T.3] Var                     0.088                                    
Group x Replication[T.4] Cov             0.494    0.171                           
Replication[T.2] x Replication[T.4] Cov  0.085    0.206                           
Replication[T.3] x Replication[T.4] Cov  0.122    0.215                           
Replication[T.4] Var                     1.481                                    
Group x Replication[T.5] Cov             0.050    0.131                           
Replication[T.2] x Replication[T.5] Cov  0.064    0.104                           
Replication[T.3] x Replication[T.5] Cov  0.107    0.118                           
Replication[T.4] x Replication[T.5] Cov  0.374    0.119                           
Replication[T.5] Var                     0.224                                    
==================================================================================

"""

In [12]:
#The authors of pymer4 recommend to add the following lines when pymer is run inside a jupyter notebook.
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [13]:
import numpy  as np
import pandas as pd
from pymer4.models import Lmer # just import the linear mixed models class 
import scipy.stats as stats 

In [14]:
eval_data = df.astype({"Graph_ID" : 'category', "Correct" : 'category', "Prediction" : 'category', "Label" : 'category', "Replication" : 'category', "system" : 'category'})

In [15]:
eval_data

,Number_of_Vertices,Number_of_Cliques,Maximum_Degree,Minimum_Degree,Average_Degree,Density,Average_Neighbor_Degree,Average_Clustering_Coefficient,Graph_ID,Prediction,Label,Confidence,Correct,Loss,Replication,system
0,10,5,5,2,4.000000,0.444444,4.000000,0.740000,0,1,1,0.859586,True,0.151305,1,emp-hgpsl
1,50,74,5,2,3.560000,0.072653,3.560000,0.156667,1,0,3,0.484204,False,3.047505,1,emp-hgpsl
2,34,39,6,2,3.705882,0.112299,3.705882,0.282353,2,2,0,0.263705,False,1.706103,1,emp-hgpsl
3,33,37,6,3,4.606061,0.143939,4.606061,0.475758,3,5,0,0.263545,False,1.803389,1,emp-hgpsl
4,52,68,6,2,3.769231,0.073906,3.769231,0.204487,4,5,4,0.263197,False,2.539231,1,emp-hgpsl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,16,13,7,3,4.500000,0.300000,4.500000,0.646726,85,2,4,0.560666,False,1.756383,5,emp-hgpsl
446,39,24,6,3,3.538462,0.093117,3.538462,0.639316,86,5,5,0.194918,True,1.635175,5,emp-hgpsl
447,28,21,6,3,3.857143,0.142857,3.857143,0.617857,87,5,1,0.340789,False,1.528071,5,emp-hgpsl
448,44,56,7,1,4.090909,0.095137,4.090909,0.249134,88,2,3,0.281418,False,1.927752,5,emp-hgpsl


In [16]:
differentMeans_model = Lmer(formula = "Loss ~ Replication + (1 | Graph_ID)", data = eval_data)

In [17]:
differentMeans_model.fit(factors = {"Replication" : ["1", "2", "3", "4", "5"]}, REML = False, summarize = False)

ValueError: Length mismatch: Expected axis has 7 elements, new values have 5 elements

In [30]:
# load df from csv
df = pd.read_csv('../df.csv', index_col=0)

In [31]:
df

,0,1,2,3,4
0,1.763881e+00,-0.015305,0.011152,0.164718,0.033078
1,1.088014e-01,0.100171,0.100171,0.100171,0.100171
2,1.933044e+02,360.000002,360.000002,360.000002,360.000002
3,1.621193e+01,-0.152790,0.111326,1.644377,0.330220
4,6.914763e-38,0.878649,0.911420,0.100971,0.741426
5,1.550634e+00,-0.211636,-0.185179,-0.031613,-0.163252
6,1.977128e+00,0.181026,0.207482,0.361049,0.229409


In [32]:
df.index = [[1], "(Intercept)", "Replication", "Replication:1", "Replication:2", "Replication:3", "Replication:4"]

In [33]:
df

,0,1,2,3,4
[1],1.763881e+00,-0.015305,0.011152,0.164718,0.033078
(Intercept),1.088014e-01,0.100171,0.100171,0.100171,0.100171
Replication,1.933044e+02,360.000002,360.000002,360.000002,360.000002
Replication:1,1.621193e+01,-0.152790,0.111326,1.644377,0.330220
Replication:2,6.914763e-38,0.878649,0.911420,0.100971,0.741426
Replication:3,1.550634e+00,-0.211636,-0.185179,-0.031613,-0.163252
Replication:4,1.977128e+00,0.181026,0.207482,0.361049,0.229409
